In [ ]:
from urllib.request import urlopen, Request
from urllib.request import urlopen
import datetime
from time import sleep
from bs4 import BeautifulSoup
import copy
import json
import sys
import csv
import pandas as pd
import numpy as np

# Link Crawl section

In [ ]:
url = 'https://www.livebitcoinnews.com/news/bitcoin-news/page/'
result = []
stop = False
for i in range(1,100):
    newurl = url+""+str(i)
    headers = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
    req = Request(url=newurl, headers=headers) 
    page = urlopen(req)
    
    soup = BeautifulSoup(page.read(), 'lxml', from_encoding="unicode")
    hrefs = SearchElement("h3", "title", soup, soloElement=False, rawElement=True);
    times = SearchElement("span", "time", soup, soloElement=False, rawElement = False);

    for href in hrefs:
        res = href.find('a')['href']
        if res == 'https://blockchain-life.com/europe/':
            continue
        result.append(res)
        
    for time in times:
        print(time)
        if "February" in time and "2019" in time:
            print("I am stopping I found critical date")          
            stop = True
            break
    if stop:
        break
    sleep(0.3)

urlsDf = pd.DataFrame(result, columns =['urls'])


In [ ]:
urlsDf.to_csv("urltoLinks.csv", encoding='utf-8', index=False)

# Content Crawl

#### Work with soup
soup.find_all('a', class_='featured-image')  
href['href'] - where href is result of find('a')  
soup.find(id="link3")
_tag.text - jadro tagu  
_tag.next_sibling - text dalsieho tagu

In [ ]:
def CheckValidUrl(url):
    if "https://" not in url:
            print("I am skipping wrong html")
            return False
    return True
        
def GetUrlSoup(url):
    headers = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
    req = Request(url=url, headers=headers) 
    page = urlopen(req)
    soup = BeautifulSoup(page.read(), 'lxml', from_encoding="unicode")
    return soup;

def GetAuthorOfPublication(soup):
    author = soup.find('span', class_="author")
    author = author.find('a').text 
    return author;

def GetDateOfPublish(soup):
    date = soup.find('span', class_="date").text
    date_time = datetime.datetime.strptime(date, '%B %d, %Y')
    return date_time

def GetTimeOfPublish(soup):
    date = soup.find('span', class_="date").text
    time = soup.find('span', class_="time").text.replace("·", "").replace(" ", "")
    datetimeText = date+" "+time
    date_time = datetime.datetime.strptime(datetimeText, '%B %d, %Y %I:%M%p')
    return date_time

def GetHeader(soup):
    header = soup.find('h1', class_="title")
    header = header.find('a').text
    return header.encode('ascii', 'ignore').decode("utf-8")

def GetSubHeaders(soup):
    subHeaders = []
    divContent = soup.find('div', class_="post-content")
    headers = divContent.find_all('h2')
    for header in headers:
        subHeaders.append((header.text).encode('ascii', 'ignore').decode("utf-8"))
    return subHeaders
    
def GetContent(soup):
    TextContent = ""
    divContent = soup.find('div', class_="post-content")
    textElements = divContent.find_all('p')
    for p in textElements:
        if "You must be" in p.text:
            continue
        TextContent = TextContent +" "+ p.text
    return TextContent.encode('ascii', 'ignore').decode("utf-8")
    
def GetTags(soup):
    tags=[]
    sectionTags = soup.find('span', class_="tags")
    stream = sectionTags.find_all('a')
    for tag in stream:
        tags.append((tag.text).encode('ascii', 'ignore').decode("utf-8"))
    return tags
    

In [ ]:
resultPd = pd.DataFrame(columns =['source','url','author','date','datetime','headertext','subheaders','contenttext','tags'])
with open('urltoLinks.csv') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        rowPd = pd.DataFrame(columns =['source','url','author','date','datetime','headertext','subheaders','contenttext','tags'])
        crawlurl=row[0]
        if not CheckValidUrl(crawlurl):
            continue;
        soup = GetUrlSoup(crawlurl)
        
        #content Crawl
        Source = "www.livebitcoinnews.com";
        Url = crawlurl;
        Author = GetAuthorOfPublication(soup);
        Date = GetDateOfPublish(soup);
        DateTime = GetTimeOfPublish(soup);
        HeaderText = GetHeader(soup);
        SubHeadersText = GetSubHeaders(soup);
        ContentText = GetContent(soup);
        Tags = GetTags(soup);
    
    
        rowPd.at[0,'source'] = Source
        rowPd.at[0,'url'] = Url
        rowPd.at[0,'author'] = Author
        rowPd.at[0,'date'] = Date
        rowPd.at[0,'datetime'] = DateTime
        rowPd.at[0,'headertext'] = HeaderText
        rowPd.at[0,'subheaders'] = SubHeadersText
        rowPd.at[0,'contenttext'] = ContentText
        rowPd.at[0,'tags'] = Tags
        print(Date)
        frames = [resultPd, rowPd]
        resultPd = pd.concat(frames)


In [ ]:
resultPd.to_csv("livebitcoinnews_data.csv", encoding='utf-8', index=False)
